In [1]:
# Initial Imports
import numpy as np
import time
import sys

In [2]:
# Creating card deck and scoring rules
cards = ["BONUS: 300"]*12 + ["BONUS: 400"]*10 + ["BONUS: 500"]*8 + ["NO DICE"]*8 + ["FILL 1000"]*6 + ["MUST BUST"]*4 + ["VENGEANCE 2500"]*4 + ["DOUBLE TROUBLE"]*2
scoring_dict = {(5,): 50, (1,): 100, (2,2,2): 200, (3,3,3): 300, (4,4,4): 400, (5,5,5): 500, (6,6,6): 600, (1,1,1): 1000, (1,2,3,4,5,6): 1500}

In [3]:
# Defining class to contain game state as attributes, actions as methods
class Game_State:
    # Instantiating base state
    def __init__(self, players):
        self.dicenum = 6
        self.draw_pile = np.random.shuffle(cards.copy())
        self.dice = []
        self.saved_dice = []
        self.turn_score = 0
        self.status = -1
        self.players_dict = dict()
        for i in range(players):
            # Potential for input for player name here
            self.players_dict[f"Player {i+1}"] = 0
            
    # Method to draw a card - also shuffles if needed and resets dice
    def draw():
        if len(self.draw_pile) < 1:
            print("New Deck!")
            self.draw_pile = np.random.shuffle(cards.copy())
        else:
            print(f"Remaining cards: {len(self.draw_pile)}")
        self.dicenum = 6
        return self.draw_pile.pop()
    
    # Method to roll dice
    def roll_dice():
        if self.status == 0:
            print("You must save some of your scoring dice before rolling again")
            print(self.dice)
            return self.dice
        self.dice = np.random.randint(low=1, high=7, size=dicenum).tolist()
        print(self.dice)
        return self.score_dice()
        
    # Method to identify possible scoring sets - called within roll_dice   
    def score_dice():   
        bust = True
        dice_counts = dict()
        for num in range(1,7):
            dice_counts[num] = self.dice.count(num)
        if set(dice_counts.values()) == {1}:
            print("Straight! 1500 points")
            bust = False
        for key in dice_counts.keys():
            if dice_counts[key] == 6:
                print(f"Two sets of {key}: {2*scoring_dict[(key, key, key)]} points")
                bust = False
            elif dice_counts[key] > 2:
                print(f"Set of {key}: {scoring_dict[(key, key, key)]} points")
                bust = False
            if (key in [1, 5]) and (dice_counts[key] > 0):
                print(f"{dice_counts[key]} {key}s: {scoring_dict[(key,)]} points each (up to {dice_counts[key]*scoring_dict[(key,)]})")
                bust = False
        if bust:
            print("Bust!")
            self.status = 3
        return self.dice
    
    # Method to save scoring dice
    def save_dice():
        if self.status == -1:
            print("No dice to save!")
            return self.dice
        while True:
            scoring_set = input("Which dice would you like to save?")
            try:
                if len(scoring_set) == 1:
                    scoring_set = [int(scoring_set)]
                else:
                    scoring_set = scoring_set.split(",")
                    scoring_set = [int(entry.strip()) for entry in scoring_set]
                break
            except:
                print("Unable to parse - please enter single scoring set of comma-separated dice values")
                return self.dice
        if tuple(scoring_set) not in scoring_dict.keys():
            print("Not a valid scoring set")
            return self.dice
        try:
            temp_dice = list()
            for die in scoring_set:
                self.dice.remove(die)
                temp_dice.append(die)
                self.dicenum -= 1
            self.saved_dice.append(temp_dice)
            self.turn_score += scoring_dict[tuple(scoring_set)]
            self.status = 1
        except:
            print("Scoring set not in dice")
        print(f"Dice: {self.dice} \nSaved Dice: {self.saved_dice} \nTurn Score: {self.turn_score}")
        if self.dicenum == 0:
            print("Fill!")
            self.status = 2
        return self.dice
    
    # Method to end turn
    def end_turn():
        if self.status < 1:
            print("You can't end your turn until you've chosen some valid scoring dice to save")
        else:
            self.players_dict[player] += self.turn_score #Can I invoke player here?
            self.status = 3
        return self.dice
    
    # Method to quit the game
    def quit_game():
        if input("Are you sure you want to quit? (y/n)").lower().strip() == "y":
            sys.exit("Thank you for playing!")
        else:
            return self.dice
    
    

In [4]:
# Status codes are used for determining the state of a player's turn:
# status -1 is not rolled yet
# status 0 is normal with no dice saved this throw
# status 1 normal with dice saved this throw
# status 2 draw again without modifying turn score
# status 3 turn is over

In [7]:
# Defining general function to get player input
def get_input(options={"0": "quit"}):
    print("What would you like to do?")
    time.sleep(2)
    while True:
        action = input(f"Choose one of the following options: {options}").lower().strip()
        if (action in options.values()) or (action in options.keys()):
            for option in options.items():
                if action in option:
                    return options_guide[option[1]]()
            break
        else:
            print("Not a valid option - try entering the number or string for one of the options provided")

In [9]:
# Options guide is the index for the functions
options_guide = {"quit": Game_State.quit_game, "draw": Game_State.draw, "roll dice": Game_State.roll_dice, "save dice": Game_State.save_dice, "end turn": Game_State.end_turn}

In [7]:
# Collecting the number of players from the user
while True:
    try:
        players = int(input("How many players? Enter a number: "))
    except:
        print("Not a valid number of players - please enter an integer")
        continue
    if (players > 1):
        break
    else:
        print("You can't play with fewer than two people!")
        continue

How many players? Enter a number:  3


In [8]:
game = Game_State(players=players)

In [9]:
game.players_dict

{'Player 1': 0, 'Player 2': 0, 'Player 3': 0}

In [12]:
# Turn loop - this contains the actual gameplay
game = Game_State(players=players)
while True:
    for player in Game_State.players_dict.keys():
        Game_State.status = -1
        Game_State.turn_score = 0
        Game_State.saved_dice.clear()
        dice.clear()
        print(f"Current scores: {players_dict}")
        print(f"{player}'s turn")
        # Draw loop
        while True:
            card = get_input({"1": "draw", "2": "end turn", "0": "quit"})
            if status == 3:
                break
            else:
                print(card)
                if card == "NO DICE":
                    print("Sorry, no dice!")
                    status = 3
                    break
                status = -1
                # Roll Dice loop
                while status < 2:
                    dice = get_input({"1": "roll dice", "2": "save dice", "3": "end turn", "0": "quit"})
                    if status > 1:
                        break
                    elif (dice == []) or (status == 1):
                        continue
                    else:
                        status = 0
            if status == 3:
                break
            status = 0
    break #temporary one-turn limit until I figure out why quit isn't working

Current scores: {'Player 1': 0, 'Player 2': 0}
Player 1's turn
What would you like to do?


Choose one of the following options: {'1': 'draw', '2': 'end turn', '0': 'quit'} 1


Remaining cards: 54
BONUS: 300
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 1


[1, 3, 2, 5, 2, 4]
1 1s: 100 points each (up to 100)
1 5s: 50 points each (up to 50)
What would you like to do?


...

Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 1


[2, 1, 6, 5, 3]
1 1s: 100 points each (up to 100)
1 5s: 50 points each (up to 50)
What would you like to do?


Choose one of the following options: {'1': 'roll dice', '2': 'save dice', '3': 'end turn', '0': 'quit'} 0


SystemExit: 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
